In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset using EIN as index
charity_df = pd.read_csv("charity_data.csv", index_col="EIN")
charity_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Store the names of all charities
charity_names = charity_df['NAME']
charity_names_df = pd.DataFrame(data=charity_names, index=charity_df.index, columns=["NAME"])
charity_names_df.head()

,NAME
EIN,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS
10553066,SOUTHSIDE ATHLETIC ASSOCIATION
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT


In [3]:
# Remove Name column
charity_df.drop(columns=["NAME"], inplace=True)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,
10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
#%%
# Check the unique value counts to see if binning is needed
classification_counts = charity_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C1728        1
C2500        1
C2170        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Create a copy of data at this point
data_charity_df = charity_df.copy()

# Determine which values to replace
replace_classifications = list(classification_counts[classification_counts < 2000].index)

# Replace in DataFrame
for classification in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification, "Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [7]:
# Check the unique value counts to see if binning is needed
apptype_counts = charity_df.APPLICATION_TYPE.value_counts()
apptype_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace
replace_apptypes = list(apptype_counts[apptype_counts < 1000].index)

# Replace in DataFrame
for apptype in replace_apptypes:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(apptype, "Other")
    
# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]), index=charity_df.index)

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10531628,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10547893,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10553066,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10556103,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Drop Special_Considerations_Y because it is a binary variable
encode_df = encode_df.drop("SPECIAL_CONSIDERATIONS_Y",1)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10531628,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10547893,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10553066,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10556103,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
# Merge one-hot encoded features and drop originals
charity_df = charity_df.merge(encode_df, left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10531628,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10547893,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10553066,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10556103,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 304       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 345
Trainable params: 345
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 126us/sample - loss: 0.6415 - accuracy: 0.6780
Epoch 2/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5738 - accuracy: 0.7263
Epoch 3/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5671 - accuracy: 0.7271
Epoch 4/100
25724/25724 [==============================] - 2s 88us/sample - loss: 0.5644 - accuracy: 0.7275
Epoch 5/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5631 - accuracy: 0.7287
Epoch 6/100
25724/25724 [==============================] - 2s 88us/sample - loss: 0.5622 - accuracy: 0.7287
Epoch 7/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5614 - accuracy: 0.7288
Epoch 8/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5609 - accuracy: 0.7290
Epoch 9/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5605 - accuracy: 0.7285
Epoc

25724/25724 [==============================] - 3s 116us/sample - loss: 0.5534 - accuracy: 0.7310
Epoch 75/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5531 - accuracy: 0.7320
Epoch 76/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5534 - accuracy: 0.7326
Epoch 77/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5533 - accuracy: 0.7317 - los
Epoch 78/100
25724/25724 [==============================] - 3s 110us/sample - loss: 0.5531 - accuracy: 0.7325
Epoch 79/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5531 - accuracy: 0.7318
Epoch 80/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5529 - accuracy: 0.7322
Epoch 81/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5533 - accuracy: 0.7321
Epoch 82/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5531 - accuracy: 0.7324
Epoch 83/100
2572

### Optimization: Target predictive accuracy >= 75%

In [16]:
# Try an additional neuron in 2nd hidden layer
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

new_nn = tf.keras.models.Sequential()

# First hidden layer
new_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
new_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
new_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
new_nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 304       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 355
Trainable params: 355
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
new_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = new_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = new_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 4s 150us/sample - loss: 0.6034 - accuracy: 0.7012
Epoch 2/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5711 - accuracy: 0.7211
Epoch 3/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5660 - accuracy: 0.7217
Epoch 4/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5640 - accuracy: 0.7229
Epoch 5/100
25724/25724 [==============================] - 3s 110us/sample - loss: 0.5629 - accuracy: 0.7259
Epoch 6/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5620 - accuracy: 0.7289
Epoch 7/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5616 - accuracy: 0.7286
Epoch 8/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5608 - accuracy: 0.7290
Epoch 9/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5602 - accuracy: 0.7

25724/25724 [==============================] - 2s 96us/sample - loss: 0.5551 - accuracy: 0.7309s - loss:
Epoch 75/100
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5549 - accuracy: 0.7306
Epoch 76/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5551 - accuracy: 0.7308
Epoch 77/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5549 - accuracy: 0.7312
Epoch 78/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5552 - accuracy: 0.7304
Epoch 79/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5549 - accuracy: 0.7308
Epoch 80/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5548 - accuracy: 0.7313
Epoch 81/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5550 - accuracy: 0.7310
Epoch 82/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5546 - accuracy: 0.7312
Epoch 83/100
25724/

The additional neuron in the 2nd layer very slightly increased accuracy.

In [18]:
# Try additional neurons in 1st hidden layer, 9 original inputs = 18 total neurons
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 4

third_nn = tf.keras.models.Sequential()

# First hidden layer
third_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
third_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
third_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
third_nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 14)                532       
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 60        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 597
Trainable params: 597
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
third_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = third_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = third_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 4s 140us/sample - loss: 0.6000 - accuracy: 0.7084
Epoch 2/100
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5670 - accuracy: 0.7280
Epoch 3/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5624 - accuracy: 0.7277
Epoch 4/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5598 - accuracy: 0.7294
Epoch 5/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5594 - accuracy: 0.7297
Epoch 6/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5586 - accuracy: 0.7304
Epoch 7/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5582 - accuracy: 0.7299
Epoch 8/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5573 - accuracy: 0.7304
Epoch 9/100
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5571 - accuracy: 0.7311


25724/25724 [==============================] - 2s 95us/sample - loss: 0.5505 - accuracy: 0.7343
Epoch 76/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5504 - accuracy: 0.7342
Epoch 77/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5504 - accuracy: 0.7341
Epoch 78/100
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5502 - accuracy: 0.7344
Epoch 79/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5504 - accuracy: 0.7343
Epoch 80/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5504 - accuracy: 0.7344
Epoch 81/100
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5502 - accuracy: 0.7341
Epoch 82/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5503 - accuracy: 0.7339
Epoch 83/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5502 - accuracy: 0.7343
Epoch 84/100
25724/25724 [====

Another slight increase. Changing activation function between hidden layers.

In [22]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 4

act_nn = tf.keras.models.Sequential()

# First hidden layer
act_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=tf.nn.leaky_relu))

# Second hidden layer
act_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
act_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
act_nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 14)                532       
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 60        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
Total params: 597
Trainable params: 597
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
act_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = act_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = act_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 130us/sample - loss: 0.6023 - accuracy: 0.7036
Epoch 2/100
25724/25724 [==============================] - 2s 93us/sample - loss: 0.5685 - accuracy: 0.7231
Epoch 3/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5654 - accuracy: 0.7254
Epoch 4/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5638 - accuracy: 0.7264
Epoch 5/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5625 - accuracy: 0.7267
Epoch 6/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5620 - accuracy: 0.7268
Epoch 7/100
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5610 - accuracy: 0.7267
Epoch 8/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5605 - accuracy: 0.7267
Epoch 9/100
25724/25724 [==============================] - 2s 93us/sample - loss: 0.5605 - accuracy: 0.7281
Epoc

25724/25724 [==============================] - 3s 107us/sample - loss: 0.5532 - accuracy: 0.7313
Epoch 76/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5531 - accuracy: 0.7308
Epoch 77/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5532 - accuracy: 0.7303
Epoch 78/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5531 - accuracy: 0.7307
Epoch 79/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5532 - accuracy: 0.7306
Epoch 80/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5530 - accuracy: 0.7309
Epoch 81/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5531 - accuracy: 0.7304
Epoch 82/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5528 - accuracy: 0.7306
Epoch 83/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5528 - accuracy: 0.7310
Epoch 84/100
25724/2572

Accuracy went down slightly with different activation functions. Next step is to try removing noisy variables from features using copy data_charity_df.

In [24]:
# Application Type and Classification were outliers in terms of number of unique values. Try removing these to optimize.
data_charity_df.drop(columns=["CLASSIFICATION"], inplace=True)
data_charity_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,
10520599,T10,Independent,ProductDev,Association,1,0,N,5000,1
10531628,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
10553066,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
10556103,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Determine which values to replace
replace_apptypes = list(apptype_counts[apptype_counts < 1000].index)

# Replace in DataFrame
for apptype in replace_apptypes:
    data_charity_df.APPLICATION_TYPE = data_charity_df.APPLICATION_TYPE.replace(apptype, "Other")
    
# Check to make sure binning was successful
data_charity_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [26]:
# Generate our categorical variable list
data_charity_cat = data_charity_df.dtypes[data_charity_df.dtypes == "object"].index.tolist()

# Fit the encoder and produce encoded DataFrame
new_encode_df = pd.DataFrame(enc.fit_transform(data_charity_df[data_charity_cat]), index=data_charity_df.index)

# Add the encoded variable names to the DataFrame
new_encode_df.columns = enc.get_feature_names(data_charity_cat)
new_encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10531628,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10547893,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10553066,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10556103,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
# Drop Special_Considerations_Y because it is a binary variable
new_encode_df = new_encode_df.drop("SPECIAL_CONSIDERATIONS_Y",1)

# Merge one-hot encoded features and drop originals
new_charity_df = data_charity_df.merge(new_encode_df, left_index=True, right_index=True)
new_charity_df = new_charity_df.drop(data_charity_cat,1)
new_charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10531628,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10547893,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10553066,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10556103,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [28]:
# Split our preprocessed data into our features and target arrays
y = new_charity_df["IS_SUCCESSFUL"].values
X = new_charity_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [29]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Use model that had 8 neurons in 1st hidden layer and 5 in 2nd hidden layer with relu in both layers
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

new_nn = tf.keras.models.Sequential()

# First hidden layer
new_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
new_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
new_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
new_nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 8)                 272       
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 6         
Total params: 323
Trainable params: 323
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
new_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = new_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = new_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 125us/sample - loss: 0.6399 - accuracy: 0.6871
Epoch 2/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5911 - accuracy: 0.7222
Epoch 3/100
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5805 - accuracy: 0.7242
Epoch 4/100
25724/25724 [==============================] - 2s 93us/sample - loss: 0.5760 - accuracy: 0.7246
Epoch 5/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5740 - accuracy: 0.7260
Epoch 6/100
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5726 - accuracy: 0.7261
Epoch 7/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5715 - accuracy: 0.7264s - loss: 0.5747 - accuracy: 0. - ETA: 0s - loss:
Epoch 8/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5705 - accuracy: 0.7265
Epoch 9/100
25724/25724 [==============================] - 2s 

25724/25724 [==============================] - 3s 100us/sample - loss: 0.5674 - accuracy: 0.7292
Epoch 75/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5670 - accuracy: 0.7294
Epoch 76/100
25724/25724 [==============================] - 3s 112us/sample - loss: 0.5669 - accuracy: 0.7297
Epoch 77/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5670 - accuracy: 0.7294
Epoch 78/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5670 - accuracy: 0.7295s - loss: 0.565
Epoch 79/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5670 - accuracy: 0.7295
Epoch 80/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5669 - accuracy: 0.7292
Epoch 81/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5671 - accuracy: 0.7290
Epoch 82/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5670 - accuracy: 0.7289
Epoch 83/1

Removing "noise variable" reduced accuracy. Try 3 hidden layers with all features included

In [32]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [33]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Define the model - deep neural net, 18 total neurons
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

three_nn = tf.keras.models.Sequential()

# First hidden layer
three_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
three_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
three_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
three_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
three_nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 8)                 304       
_________________________________________________________________
dense_19 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 6         
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
three_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = three_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = three_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 4s 143us/sample - loss: 0.6155 - accuracy: 0.6875
Epoch 2/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5727 - accuracy: 0.7262
Epoch 3/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5674 - accuracy: 0.7254
Epoch 4/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5644 - accuracy: 0.7269
Epoch 5/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5635 - accuracy: 0.7268
Epoch 6/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5626 - accuracy: 0.7262
Epoch 7/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5616 - accuracy: 0.7282
Epoch 8/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5610 - accuracy: 0.7287
Epoch 9/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5608 - accuracy: 0.7269

25724/25724 [==============================] - 3s 117us/sample - loss: 0.5533 - accuracy: 0.7311
Epoch 76/100
25724/25724 [==============================] - 3s 118us/sample - loss: 0.5532 - accuracy: 0.7311
Epoch 77/100
25724/25724 [==============================] - 3s 120us/sample - loss: 0.5534 - accuracy: 0.7322
Epoch 78/100
25724/25724 [==============================] - 3s 115us/sample - loss: 0.5532 - accuracy: 0.7317
Epoch 79/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5535 - accuracy: 0.7318
Epoch 80/100
25724/25724 [==============================] - 3s 121us/sample - loss: 0.5534 - accuracy: 0.7317
Epoch 81/100
25724/25724 [==============================] - 3s 118us/sample - loss: 0.5532 - accuracy: 0.7312
Epoch 82/100
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5535 - accuracy: 0.7324
Epoch 83/100
25724/25724 [==============================] - 3s 117us/sample - loss: 0.5531 - accuracy: 0.7320
Epoch 84/100
25724/2572